# Analise e processamento dos dados recebidos

# CONFIGURAR LOG DO SISTEMA

In [326]:
import sys
from pathlib import Path

# Adiciona o diretório src ao path do Python
src_path = Path.cwd().parent / 'src'
if str(src_path) not in sys.path:
	sys.path.insert(0, str(src_path))

import logging
from datetime import datetime
from fiap import LoggerManager

log_path = Path.cwd().parent / 'logs'
LoggerManager(log_path=log_path, base_filename='data_processing')
# Dessa forma usando o logging padrão ele salva no arquivo
logging.info(f'{"=" * 50}')
logging.info(f'Início do processamento: {datetime.now()}')

2026-02-15 21:28:20,728 - INFO - Logger initialized: g:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\logs\2026-02-15_data_processing.log
2026-02-15 21:28:20,731 - INFO - ==================================================
2026-02-15 21:28:20,731 - INFO - Início do processamento: 2026-02-15 21:28:20.731364


# Transformar o arquivo xlsx em csv (apenas 2024)

In [327]:
from fiap.utils import excel_to_csv

logging.info('Iniciando a conversão de Excel para CSV (2024)')
xlxs_path = Path.cwd().parent / 'data/xlxs/BASE DE DADOS PEDE 2024 - DATATHON.xlsx'
output_path = Path.cwd().parent / 'data/csv'
excel_to_csv(
	excel_path=xlxs_path,
	output_folder=output_path,
)

2026-02-15 21:28:20,752 - INFO - Iniciando a conversão de Excel para CSV (2024)
2026-02-15 21:28:21,402 - INFO - Processando arquivo: BASE DE DADOS PEDE 2024 - DATATHON.xlsx
2026-02-15 21:28:21,402 - INFO - Total de planilhas encontradas: 3

2026-02-15 21:28:21,878 - INFO - ✓ Planilha 'PEDE2022' convertida para: PEDE2022.csv
2026-02-15 21:28:21,878 - INFO -   Dimensões: 860 linhas x 42 colunas
2026-02-15 21:28:22,392 - INFO - ✓ Planilha 'PEDE2023' convertida para: PEDE2023.csv
2026-02-15 21:28:22,392 - INFO -   Dimensões: 1014 linhas x 48 colunas
2026-02-15 21:28:23,694 - INFO - ✓ Planilha 'PEDE2024' convertida para: PEDE2024.csv
2026-02-15 21:28:23,694 - INFO -   Dimensões: 1156 linhas x 50 colunas
2026-02-15 21:28:23,694 - INFO - 
✓ Conversão concluída! 3 arquivo(s) CSV criado(s) em: g:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\data\csv


# Carregar dados de 2024

In [328]:
import pandas as pd

# Caminho para o arquivo CSV de 2024
csv_path = Path.cwd().parent / 'data/csv/PEDE2024.csv'

logging.info(f'Lendo arquivo: {csv_path.name}')
df = pd.read_csv(csv_path)

logging.info(f'DataFrame carregado: {df.shape[0]} linhas, {df.shape[1]} colunas')

# nome das colunas em minusculo para padronizar
df.columns = df.columns.str.lower()
df.head()

2026-02-15 21:28:23,722 - INFO - Lendo arquivo: PEDE2024.csv
2026-02-15 21:28:23,847 - INFO - DataFrame carregado: 1156 linhas, 50 colunas


,ra,fase,inde 2024,pedra 2024,turma,nome anonimizado,data de nasc,idade,gênero,ano ingresso,...,ipv,ian,fase ideal,defasagem,destaque ieg,destaque ida,destaque ipv,escola,ativo/ inativo,ativo/ inativo.1
0,RA-1275,ALFA,7.611366666700001,Ametista,ALFA A - G0/G1,Aluno-1275,2016-07-28 00:00:00,8,Masculino,2024,...,5.446667,10.0,ALFA (1° e 2° ano),0,NaN,NaN,NaN,EE Chácara Florida II,Cursando,Cursando
1,RA-1276,ALFA,8.002866666700001,Topázio,ALFA A - G0/G1,Aluno-1276,2016-10-16 00:00:00,8,Feminino,2024,...,7.050000,10.0,ALFA (1° e 2° ano),0,NaN,NaN,NaN,EE Chácara Florida II,Cursando,Cursando
2,RA-1277,ALFA,7.952200000100001,Ametista,ALFA A - G0/G1,Aluno-1277,2016-08-16 00:00:00,8,Masculino,2024,...,7.046667,10.0,ALFA (1° e 2° ano),0,NaN,NaN,NaN,EE Dom Pedro Villas Boas de Souza,Cursando,Cursando
3,RA-868,ALFA,7.156366666600001,Ametista,ALFA A - G0/G1,Aluno-868,2015-11-08 00:00:00,8,Masculino,2023,...,7.213333,5.0,Fase 1 (3° e 4° ano),-1,NaN,NaN,NaN,EE Chácara Florida II,Cursando,Cursando
4,RA-1278,ALFA,5.444199999900001,Quartzo,ALFA A - G0/G1,Aluno-1278,2015-03-22 00:00:00,9,Masculino,2024,...,4.173333,5.0,Fase 1 (3° e 4° ano),-1,NaN,NaN,NaN,EM Etelvina Delfim Simões,Cursando,Cursando


# Exploração inicial dos dados

In [329]:
logging.info(f'{"=" * 50}')
logging.info(f'Colunas do dataset: {list(df.columns)}')
logging.info(f'Tipos de dados:\n{df.dtypes}')
logging.info(f'Valores nulos:\n{df.isnull().sum()}')
logging.info(f'\nEstatísticas descritivas:\n{df.describe()}')

# Exemplo de um aluno
logging.info(f'{"=" * 50}')
logging.info('Exemplo de registro de aluno:')
aluno = df.loc[0].to_dict()
for k, v in aluno.items():
	logging.info(f'{k}: {v}')

2026-02-15 21:28:23,901 - INFO - ==================================================
2026-02-15 21:28:23,909 - INFO - Colunas do dataset: ['ra', 'fase', 'inde 2024', 'pedra 2024', 'turma', 'nome anonimizado', 'data de nasc', 'idade', 'gênero', 'ano ingresso', 'instituição de ensino', 'pedra 20', 'pedra 21', 'pedra 22', 'pedra 23', 'inde 22', 'inde 23', 'cg', 'cf', 'ct', 'nº av', 'avaliador1', 'rec av1', 'avaliador2', 'rec av2', 'avaliador3', 'avaliador4', 'avaliador5', 'avaliador6', 'iaa', 'ieg', 'ips', 'ipp', 'rec psicologia', 'ida', 'mat', 'por', 'ing', 'indicado', 'atingiu pv', 'ipv', 'ian', 'fase ideal', 'defasagem', 'destaque ieg', 'destaque ida', 'destaque ipv', 'escola', 'ativo/ inativo', 'ativo/ inativo.1']
2026-02-15 21:28:23,914 - INFO - Tipos de dados:
ra                        object
fase                      object
inde 2024                 object
pedra 2024                object
turma                     object
nome anonimizado          object
data de nasc              obj

# Limpeza de dados - Remover colunas desnecessárias

In [330]:
# Colunas que serão removidas (ajuste conforme necessário)
unused_columns = [
	col
	for col in df.columns
	if col.startswith('nome')
	or col.startswith('data')
	or col.startswith('ano')
	or col.startswith('ra')
	or col.startswith('avaliador')
	or col.startswith('nº')
	or col.startswith('turma')
]

if unused_columns:
	logging.info(f'Removendo {len(unused_columns)} colunas desnecessárias: {unused_columns}')
	df = df.drop(columns=unused_columns)
	logging.info(f'DataFrame após limpeza: {df.shape[0]} linhas, {df.shape[1]} colunas')
else:
	logging.info('Nenhuma coluna desnecessária encontrada')

df.head()

2026-02-15 21:28:24,113 - INFO - Removendo 12 colunas desnecessárias: ['ra', 'turma', 'nome anonimizado', 'data de nasc', 'ano ingresso', 'nº av', 'avaliador1', 'avaliador2', 'avaliador3', 'avaliador4', 'avaliador5', 'avaliador6']
2026-02-15 21:28:24,118 - INFO - DataFrame após limpeza: 1156 linhas, 38 colunas


,fase,inde 2024,pedra 2024,idade,gênero,instituição de ensino,pedra 20,pedra 21,pedra 22,pedra 23,...,ipv,ian,fase ideal,defasagem,destaque ieg,destaque ida,destaque ipv,escola,ativo/ inativo,ativo/ inativo.1
0,ALFA,7.611366666700001,Ametista,8,Masculino,Pública,NaN,NaN,NaN,NaN,...,5.446667,10.0,ALFA (1° e 2° ano),0,NaN,NaN,NaN,EE Chácara Florida II,Cursando,Cursando
1,ALFA,8.002866666700001,Topázio,8,Feminino,Pública,NaN,NaN,NaN,NaN,...,7.050000,10.0,ALFA (1° e 2° ano),0,NaN,NaN,NaN,EE Chácara Florida II,Cursando,Cursando
2,ALFA,7.952200000100001,Ametista,8,Masculino,Pública,NaN,NaN,NaN,NaN,...,7.046667,10.0,ALFA (1° e 2° ano),0,NaN,NaN,NaN,EE Dom Pedro Villas Boas de Souza,Cursando,Cursando
3,ALFA,7.156366666600001,Ametista,8,Masculino,Pública,NaN,NaN,NaN,Topázio,...,7.213333,5.0,Fase 1 (3° e 4° ano),-1,NaN,NaN,NaN,EE Chácara Florida II,Cursando,Cursando
4,ALFA,5.444199999900001,Quartzo,9,Masculino,Pública,NaN,NaN,NaN,NaN,...,4.173333,5.0,Fase 1 (3° e 4° ano),-1,NaN,NaN,NaN,EM Etelvina Delfim Simões,Cursando,Cursando


In [331]:
print(df['ativo/ inativo'].value_counts())
print(df['ativo/ inativo.1'].value_counts())

ativo/ inativo
Cursando    1156
Name: count, dtype: int64
ativo/ inativo.1
Cursando    1156
Name: count, dtype: int64


In [332]:
# como estao todos cursando vou remover essas colunas
df = df.drop(columns=['ativo/ inativo', 'ativo/ inativo.1'])

In [333]:
logging.info(f'{"=" * 50}')
logging.info(f'Processamento concluído: {datetime.now()}')
logging.info(f'Dataset final: {df.shape[0]} linhas, {df.shape[1]} colunas')

2026-02-15 21:28:24,203 - INFO - ==================================================
2026-02-15 21:28:24,205 - INFO - Processamento concluído: 2026-02-15 21:28:24.205213
2026-02-15 21:28:24,207 - INFO - Dataset final: 1156 linhas, 36 colunas


# REMOVER VALORES NULOS

In [334]:
# Remover colunas que tenham mais de 30% de valores nulos
threshold = len(df) * 0.3
cols_to_drop = df.columns[df.isnull().sum() > threshold]
if len(cols_to_drop) > 0:
	logging.info(
		f'Removendo {len(cols_to_drop)} colunas com mais de 30% de valores nulos: {list(cols_to_drop)}'
	)
	df = df.drop(columns=cols_to_drop)
	logging.info(
		f'DataFrame após remoção de colunas nulas: {df.shape[0]} linhas, {df.shape[1]} colunas'
	)

2026-02-15 21:28:24,250 - INFO - Removendo 18 colunas com mais de 30% de valores nulos: ['pedra 20', 'pedra 21', 'pedra 22', 'pedra 23', 'inde 22', 'inde 23', 'cg', 'cf', 'ct', 'rec av1', 'rec av2', 'rec psicologia', 'ing', 'indicado', 'atingiu pv', 'destaque ieg', 'destaque ida', 'destaque ipv']
2026-02-15 21:28:24,255 - INFO - DataFrame após remoção de colunas nulas: 1156 linhas, 18 colunas


In [335]:
df.head(1)

,fase,inde 2024,pedra 2024,idade,gênero,instituição de ensino,iaa,ieg,ips,ipp,ida,mat,por,ipv,ian,fase ideal,defasagem,escola
0,ALFA,7.611366666700001,Ametista,8,Masculino,Pública,10.002,8.666667,6.26,5.625,8.0,10.0,6.0,5.446667,10.0,ALFA (1° e 2° ano),0,EE Chácara Florida II


# ENCODING

In [336]:
df.dtypes

fase                      object
inde 2024                 object
pedra 2024                object
idade                      int64
gênero                    object
instituição de ensino     object
iaa                      float64
ieg                      float64
ips                      float64
ipp                      float64
ida                      float64
mat                      float64
por                      float64
ipv                      float64
ian                      float64
fase ideal                object
defasagem                  int64
escola                    object
dtype: object

In [337]:
df.loc[1]

fase                                      ALFA
inde 2024                    8.002866666700001
pedra 2024                             Topázio
idade                                        8
gênero                                Feminino
instituição de ensino                  Pública
iaa                                     10.002
ieg                                   9.333333
ips                                       3.76
ipp                                        7.5
ida                                        8.0
mat                                       10.0
por                                        6.0
ipv                                       7.05
ian                                       10.0
fase ideal                  ALFA (1° e 2° ano)
defasagem                                    0
escola                   EE Chácara Florida II
Name: 1, dtype: object

# Padronizar Fases

In [338]:
logging.info(f'{"=" * 50}')
logging.info('Contagem de alunos por fase:')
for f, q in df['fase'].value_counts().items():
	logging.info(f'Fase: {f}, Quantidade: {q}')

2026-02-15 21:28:24,374 - INFO - ==================================================
2026-02-15 21:28:24,377 - INFO - Contagem de alunos por fase:
2026-02-15 21:28:24,381 - INFO - Fase: ALFA, Quantidade: 196
2026-02-15 21:28:24,383 - INFO - Fase: 9, Quantidade: 38
2026-02-15 21:28:24,384 - INFO - Fase: 7E, Quantidade: 25
2026-02-15 21:28:24,385 - INFO - Fase: 8E, Quantidade: 23
2026-02-15 21:28:24,386 - INFO - Fase: 4M, Quantidade: 18
2026-02-15 21:28:24,387 - INFO - Fase: 4B, Quantidade: 17
2026-02-15 21:28:24,388 - INFO - Fase: 2L, Quantidade: 16
2026-02-15 21:28:24,388 - INFO - Fase: 1G, Quantidade: 16
2026-02-15 21:28:24,390 - INFO - Fase: 3D, Quantidade: 16
2026-02-15 21:28:24,394 - INFO - Fase: 3U, Quantidade: 16
2026-02-15 21:28:24,396 - INFO - Fase: 4F, Quantidade: 16
2026-02-15 21:28:24,398 - INFO - Fase: 2M, Quantidade: 16
2026-02-15 21:28:24,399 - INFO - Fase: 3K, Quantidade: 16
2026-02-15 21:28:24,400 - INFO - Fase: 2K, Quantidade: 16
2026-02-15 21:28:24,401 - INFO - Fase: 3

In [339]:
logging.info('Contagem de alunos por fase ideal:')
for f, q in df['fase ideal'].value_counts().items():
	logging.info(f'Fase ideal: {f}, Quantidade: {q}')

2026-02-15 21:28:24,546 - INFO - Contagem de alunos por fase ideal:
2026-02-15 21:28:24,550 - INFO - Fase ideal: Fase 2 (5° e 6° ano), Quantidade: 281
2026-02-15 21:28:24,550 - INFO - Fase ideal: Fase 3 (7° e 8° ano), Quantidade: 233
2026-02-15 21:28:24,550 - INFO - Fase ideal: Fase 1 (3° e 4° ano), Quantidade: 182
2026-02-15 21:28:24,550 - INFO - Fase ideal: Fase 8 (Universitários), Quantidade: 102
2026-02-15 21:28:24,550 - INFO - Fase ideal: Fase 5 (1° EM), Quantidade: 96
2026-02-15 21:28:24,550 - INFO - Fase ideal: Fase 4 (9° ano), Quantidade: 90
2026-02-15 21:28:24,557 - INFO - Fase ideal: Fase 6 (2° EM), Quantidade: 70
2026-02-15 21:28:24,559 - INFO - Fase ideal: Fase 7 (3° EM), Quantidade: 53
2026-02-15 21:28:24,560 - INFO - Fase ideal: ALFA (1° e 2° ano), Quantidade: 49


Vamos padronizar para ALFA e fase1-8

In [340]:
# Padronização da coluna "fase"
def padronizar_fase(valor):
	v = str(valor).strip().lower()
	v = v.replace('fase ', '')  # Remove o prefixo "fase " se existir
	if v.startswith('alfa'):
		return 'alfa'
	# Extrai o primeiro número encontrado (1-8) para as fases
	import re

	match = re.search(r'[1-8]', v)
	if match:
		return f'{match.group(0)}'
	return v


# Aplicar padronização
logging.info('Padronizar Fase')
df['fase'] = df['fase'].apply(padronizar_fase)

logging.info('Padronização de fases concluída.')
logging.info('Contagem de alunos por fase padronizada:')
for f, q in df['fase'].value_counts().items():
	logging.info(f'Fase: {f}, Quantidade: {q}')

# fase ideal
logging.info('Padronizar Fase Ideal')
df['fase ideal'] = df['fase ideal'].apply(padronizar_fase)

logging.info('Padronização de fase ideal concluída.')
logging.info('Contagem de alunos por fase ideal padronizada:')
for f, q in df['fase ideal'].value_counts().items():
	logging.info(f'Fase: {f}, Quantidade: {q}')

2026-02-15 21:28:24,598 - INFO - Padronizar Fase
2026-02-15 21:28:24,603 - INFO - Padronização de fases concluída.
2026-02-15 21:28:24,604 - INFO - Contagem de alunos por fase padronizada:
2026-02-15 21:28:24,606 - INFO - Fase: 3, Quantidade: 211
2026-02-15 21:28:24,609 - INFO - Fase: alfa, Quantidade: 196
2026-02-15 21:28:24,610 - INFO - Fase: 1, Quantidade: 185
2026-02-15 21:28:24,611 - INFO - Fase: 2, Quantidade: 185
2026-02-15 21:28:24,613 - INFO - Fase: 4, Quantidade: 115
2026-02-15 21:28:24,614 - INFO - Fase: 5, Quantidade: 100
2026-02-15 21:28:24,616 - INFO - Fase: 8, Quantidade: 64
2026-02-15 21:28:24,618 - INFO - Fase: 9, Quantidade: 38
2026-02-15 21:28:24,619 - INFO - Fase: 7, Quantidade: 37
2026-02-15 21:28:24,620 - INFO - Fase: 6, Quantidade: 25
2026-02-15 21:28:24,622 - INFO - Padronizar Fase Ideal
2026-02-15 21:28:24,629 - INFO - Padronização de fase ideal concluída.
2026-02-15 21:28:24,630 - INFO - Contagem de alunos por fase ideal padronizada:
2026-02-15 21:28:24,635 - 

## Remover a escola pois assim pode ser utilizado para novas escolas

In [341]:
logging.info('Remover Escola pois assim pode ser usado para novas escolas')
df.drop(columns=['escola'], inplace=True)

2026-02-15 21:28:24,719 - INFO - Remover Escola pois assim pode ser usado para novas escolas


# Tratar dados String

In [342]:
logging.info('Tratando Strings')
logging.info(df.select_dtypes(include=['object', 'string']).columns)

2026-02-15 21:28:24,879 - INFO - Tratando Strings
2026-02-15 21:28:24,882 - INFO - Index(['fase', 'inde 2024', 'pedra 2024', 'gênero', 'instituição de ensino',
       'fase ideal'],
      dtype='object')


## Fase

In [343]:
logging.info('Tratamento de fase')
mapa_fase = {'alfa': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
logging.info(df['fase'].value_counts())
df['fase'] = df['fase'].astype(str).map(mapa_fase)
logging.info(df['fase ideal'].value_counts())
df['fase ideal'] = df['fase ideal'].astype(str).map(mapa_fase)

logging.info("Como a fase ideal é algo que não vamos ter no futuro e teremos que prever, vamos remover essa coluna do dataset para evitar vazamento de dados")
df = df.drop(columns=['fase ideal'])

2026-02-15 21:28:24,940 - INFO - Tratamento de fase
2026-02-15 21:28:24,945 - INFO - fase
3       211
alfa    196
1       185
2       185
4       115
5       100
8        64
9        38
7        37
6        25
Name: count, dtype: int64
2026-02-15 21:28:24,950 - INFO - fase ideal
2       281
3       233
1       182
8       102
5        96
4        90
6        70
7        53
alfa     49
Name: count, dtype: int64
2026-02-15 21:28:24,956 - INFO - Como a fase ideal é algo que não vamos ter no futuro e teremos que prever, vamos remover essa coluna do dataset para evitar vazamento de dados


## Inde 24

In [344]:
logging.info(
	'Inde 24 é uma ponderação de outros indicadores (IAN, IDA, IEG, IAA, IPS, IPP e IPV), '
)
logging.info('Pedra 2024 é baseada no Inde 24')
logging.info(
	'Como vamos utilizar os outros indicadores para criar o modelo vamos remover essas duas colunas'
)
df.drop(columns=['pedra 2024', 'inde 2024'], inplace=True)
logging.info("Colunas 'pedra 2024' e 'inde 24' removidas")
logging.info('Colunas restantes: {}'.format(list(df.columns)))

2026-02-15 21:28:24,977 - INFO - Inde 24 é uma ponderação de outros indicadores (IAN, IDA, IEG, IAA, IPS, IPP e IPV), 
2026-02-15 21:28:24,979 - INFO - Pedra 2024 é baseada no Inde 24
2026-02-15 21:28:24,981 - INFO - Como vamos utilizar os outros indicadores para criar o modelo vamos remover essas duas colunas
2026-02-15 21:28:24,984 - INFO - Colunas 'pedra 2024' e 'inde 24' removidas
2026-02-15 21:28:24,985 - INFO - Colunas restantes: ['fase', 'idade', 'gênero', 'instituição de ensino', 'iaa', 'ieg', 'ips', 'ipp', 'ida', 'mat', 'por', 'ipv', 'ian', 'defasagem']


In [345]:
df.select_dtypes(include=['object', 'string']).columns

Index(['gênero', 'instituição de ensino'], dtype='object')

# Gênero

In [346]:
logging.info('Utilizar Encoding para a coluna Gênero')
logging.info(df['gênero'].value_counts())
# Criar colunas manualmente
df['genero_f'] = (df['gênero'] == 'Feminino').astype(int)
df['genero_m'] = (df['gênero'] == 'Masculino').astype(int)

# Remover a coluna original
df = df.drop(columns=['gênero'])
df.head(1)

2026-02-15 21:28:25,025 - INFO - Utilizar Encoding para a coluna Gênero
2026-02-15 21:28:25,028 - INFO - gênero
Feminino     623
Masculino    533
Name: count, dtype: int64


,fase,idade,instituição de ensino,iaa,ieg,ips,ipp,ida,mat,por,ipv,ian,defasagem,genero_f,genero_m
0,0,8,Pública,10.002,8.666667,6.26,5.625,8.0,10.0,6.0,5.446667,10.0,0,0,1


# Instituição de ensino

In [347]:
import json
import os

logging.info('Instituição de ensino original:')
logging.info(df['instituição de ensino'].value_counts())

# Normalizar para minúsculas
df['instituição_norm'] = df['instituição de ensino'].str.lower()

# Criar manualmente o map
# Aqui você decide manualmente a ordem que quer
map_instituicao = {
	'pública': 1,
	'privada - programa de apadrinhamento': 2,
	'privada': 3,
	'privada *parcerias com bolsa 100%': 4,
	'bolsista universitário *formado (a)': 5,
	'privada - pagamento por *empresa parceira': 6,
	'concluiu o 3º em': 7,
}

logging.info('Mapeamento manual criado:')
logging.info(map_instituicao)

# Criar colunas One-Hot manualmente
for cat, idx in map_instituicao.items():
	col_name = f'instituição_tipo_{idx}'
	df[col_name] = (df['instituição_norm'] == cat).astype(int)

# Remover colunas originais e temporárias
df = df.drop(columns=['instituição de ensino', 'instituição_norm'], errors='ignore')

logging.info('DataFrame final com colunas One-Hot:')
logging.info(df)

# Salvar o map em JSON na pasta docs
docs_path = Path.cwd().parent / 'docs'
os.makedirs(docs_path, exist_ok=True)
with open(docs_path / 'map_instituicao_ensino.json', 'w', encoding='utf-8') as f:
	json.dump(map_instituicao, f, ensure_ascii=False, indent=4)

logging.info("Arquivo map_instituicao_ensino.json salvo em 'docs/'")

2026-02-15 21:28:25,080 - INFO - Instituição de ensino original:
2026-02-15 21:28:25,082 - INFO - instituição de ensino
Pública                                      913
Privada - Programa de Apadrinhamento          95
Privada                                       76
Privada *Parcerias com Bolsa 100%             41
Bolsista Universitário *Formado (a)           13
Privada - Pagamento por *Empresa Parceira      9
Concluiu o 3º EM                               7
Privada - Programa de apadrinhamento           1
Name: count, dtype: int64
2026-02-15 21:28:25,086 - INFO - Mapeamento manual criado:
2026-02-15 21:28:25,087 - INFO - {'pública': 1, 'privada - programa de apadrinhamento': 2, 'privada': 3, 'privada *parcerias com bolsa 100%': 4, 'bolsista universitário *formado (a)': 5, 'privada - pagamento por *empresa parceira': 6, 'concluiu o 3º em': 7}
2026-02-15 21:28:25,097 - INFO - DataFrame final com colunas One-Hot:
2026-02-15 21:28:25,098 - INFO -       fase  idade     iaa       ieg   ips 

In [348]:
for ind in df.head(1):
	logging.info(f'{ind}: {df[ind].iloc[0]}')

2026-02-15 21:28:25,253 - INFO - fase: 0
2026-02-15 21:28:25,255 - INFO - idade: 8
2026-02-15 21:28:25,257 - INFO - iaa: 10.002
2026-02-15 21:28:25,260 - INFO - ieg: 8.6666666665
2026-02-15 21:28:25,261 - INFO - ips: 6.26
2026-02-15 21:28:25,262 - INFO - ipp: 5.625
2026-02-15 21:28:25,264 - INFO - ida: 8.0
2026-02-15 21:28:25,265 - INFO - mat: 10.0
2026-02-15 21:28:25,266 - INFO - por: 6.0
2026-02-15 21:28:25,267 - INFO - ipv: 5.446666667
2026-02-15 21:28:25,268 - INFO - ian: 10.0
2026-02-15 21:28:25,269 - INFO - defasagem: 0
2026-02-15 21:28:25,270 - INFO - genero_f: 0
2026-02-15 21:28:25,272 - INFO - genero_m: 1
2026-02-15 21:28:25,273 - INFO - instituição_tipo_1: 1
2026-02-15 21:28:25,274 - INFO - instituição_tipo_2: 0
2026-02-15 21:28:25,276 - INFO - instituição_tipo_3: 0
2026-02-15 21:28:25,277 - INFO - instituição_tipo_4: 0
2026-02-15 21:28:25,277 - INFO - instituição_tipo_5: 0
2026-02-15 21:28:25,280 - INFO - instituição_tipo_6: 0
2026-02-15 21:28:25,281 - INFO - instituição_tip

# Tratamendo dos valores nulos restantes

In [349]:
# Contar nulos por coluna
nulos_por_coluna = df.isna().sum()

# Filtrar só colunas que têm algum nulo
colunas_com_nulo = nulos_por_coluna[nulos_por_coluna > 0]

logging.info('Colunas com valores nulos:')
logging.info(colunas_com_nulo)
logging.info('Preenchendo valores nulos com a média das colunas numéricas')
for col in ['iaa', 'ips', 'ipp', 'ida', 'mat', 'por', 'ipv']:
	df[col].fillna(df[col].mean(), inplace=True)

logging.info('Valores nulos preenchidos.')

2026-02-15 21:28:25,311 - INFO - Colunas com valores nulos:
2026-02-15 21:28:25,315 - INFO - iaa    102
ips    102
ipp    102
ida    101
mat    105
por    106
ipv    102
dtype: int64
2026-02-15 21:28:25,319 - INFO - Preenchendo valores nulos com a média das colunas numéricas
C:\Users\DELL\AppData\Local\Temp\ipykernel_15556\2230095274.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
2026-02-15 21:28:25,338 - INFO - Valores nulos preenchidos.


In [350]:
df.isna().sum()

fase                  0
idade                 0
iaa                   0
ieg                   0
ips                   0
ipp                   0
ida                   0
mat                   0
por                   0
ipv                   0
ian                   0
defasagem             0
genero_f              0
genero_m              0
instituição_tipo_1    0
instituição_tipo_2    0
instituição_tipo_3    0
instituição_tipo_4    0
instituição_tipo_5    0
instituição_tipo_6    0
instituição_tipo_7    0
dtype: int64

# Salvar em data

In [351]:
data_path = Path.cwd().parent / 'data'
os.makedirs(data_path, exist_ok=True)
df.to_csv(data_path / 'processed_data.csv', index=False)
logging.info("DataFrame processado salvo em 'data/processed_data.csv'")

2026-02-15 21:28:25,652 - INFO - DataFrame processado salvo em 'data/processed_data.csv'


In [352]:
df.columns

Index(['fase', 'idade', 'iaa', 'ieg', 'ips', 'ipp', 'ida', 'mat', 'por', 'ipv',
       'ian', 'defasagem', 'genero_f', 'genero_m', 'instituição_tipo_1',
       'instituição_tipo_2', 'instituição_tipo_3', 'instituição_tipo_4',
       'instituição_tipo_5', 'instituição_tipo_6', 'instituição_tipo_7'],
      dtype='object')